<a href="https://colab.research.google.com/github/GaryPython/Cathay_LLM/blob/main/R3/R3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三次作業

## Basline
將第一次作業的 QA 檢索聊天機器人，重新透過 LangChain 進行實作

###  1.Document Loading

In [1]:
!pip install langchain_huggingface
!pip install bitsandbytes
!pip install ipykernel ipywidgets langchain PyMuPDF chromadb sentence-transformers llama-cpp-python openai langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
pdf_files="/content/basketball.pdf"

In [3]:
pdf_files

'/content/basketball.pdf'

In [5]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader(pdf_files)
pages = loader.load()
pages

[Document(metadata={'source': '/content/basketball.pdf', 'file_path': '/content/basketball.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': 'User', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20240312114435+08'00'", 'modDate': "D:20240312114435+08'00'", 'trapped': ''}, page_content='112 學年度學生籃球爭霸賽競賽規程 \n主辦單位：體育室  \n一、主旨：藉體育活動聯絡同學之情誼，提倡校園籃球運動風氣，並提升對反菸毒議題的重視促進\n身心健康，特舉辦本比賽。 \n二、比賽時間：113 年 4 月 8 日至 5 月 3 日。  \n三、比賽地點：本校室外籃球場。\n（※遇雨天延期舉行，時間依體育室公告為準）  \n四、參加單位：  \n   （一）本校教學單位之各所、系、科學生均得聯合組隊參加。  \n   （二）由每一教學單位在籍學生聯合組隊參加，每組以一隊為限。  \n五、比賽分組及報名資格：  \n   （一）比賽分為男子組和女子組。  \n   （二）組隊之成員必須為同所、系、科學生聯合組隊各所系科最多2 隊，各隊報名人數以 18 人 \n 為上限；男子組男生不足時得以女生代替。  \n六、報名時間及抽籤時間：  \n   （一）自即日起至 113 年 3 月 22 日(五)中午 12 時止，將報名表繕打後寄至體育室趙于葳老\n師信箱（ryukyu919@ntub.edu.tw）\n，逾期不予受理。若有相關問題，請電洽 2322-6177。  \n   （二）報名表請至本校體育室網頁下載。  \n   （三）抽籤：113 年 3 月 25 日(一) 12 時 15 分於體育室舉行，逾時未到者由大會代抽，不\n得異議。  \n七、比賽制度及規則：  \n （一）比賽制度：

### 2.Splitting

In [11]:
'''
from langchain.document_loaders import PyPDFLoader

fiel_dir = "PDF_DIRECTORY"


loader = PyPDFLoader(fiel_dir)
pages = loader.load()

'''
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


pdf_files="/content/basketball.pdf"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20) #分割一段話100字元，最後20個字元可以傳到下一段話
all_texts = []
for pdf_file in pdf_files:
    loader = PyMuPDFLoader(pdf_files)
    PDF_data = loader.load()
    all_splits = text_splitter.split_documents(PDF_data)
    all_texts.extend(all_splits)

all_texts

[Document(metadata={'source': '/content/basketball.pdf', 'file_path': '/content/basketball.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': 'User', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20240312114435+08'00'", 'modDate': "D:20240312114435+08'00'", 'trapped': ''}, page_content='112 學年度學生籃球爭霸賽競賽規程 \n主辦單位：體育室  \n一、主旨：藉體育活動聯絡同學之情誼，提倡校園籃球運動風氣，並提升對反菸毒議題的重視促進\n身心健康，特舉辦本比賽。'),
 Document(metadata={'source': '/content/basketball.pdf', 'file_path': '/content/basketball.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': 'User', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20240312114435+08'00'", 'modDate': "D:20240312114435+08'00'", 'trapped': ''}, page_content='身心健康，特舉辦本比賽。 \n二、比賽時間：113 年 4 月 8 日至 5 月 3 日。  \n三、比賽地點：本校室外籃球場。\n（※遇雨天延期舉行，時間依體育室公告為準）  \n四、參加單位：'),
 Document(metadata={'s

### 3.1 Embedding

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity


# 初始化 Embedding 模型
embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    #model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True})


'''
# 將字句轉換為向量
a = embedding_func.embed_query('突襲式發表！蘋果推 2 款 M3 MacBook Air，強調 AI 、遊戲效能皆強化')
b = embedding_func.embed_query('蘋果最新M3版MacBook Air突襲登場！6亮點下放1技術不漲價 M2版還降3000元')

# 計算相似度
cosine_similarity([a], [b])
'''

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


"\n# 將字句轉換為向量\na = embedding_func.embed_query('突襲式發表！蘋果推 2 款 M3 MacBook Air，強調 AI 、遊戲效能皆強化')\nb = embedding_func.embed_query('蘋果最新M3版MacBook Air突襲登場！6亮點下放1技術不漲價 M2版還降3000元')\n\n# 計算相似度\ncosine_similarity([a], [b])\n"

### 3.2 Vectore Store


In [12]:
from langchain.vectorstores import Chroma


# load it into Chroma
db = Chroma.from_documents(all_texts, embedding_func)

# query it
#query = "什麼是 LLM 模型？"
query = "請問臺北商業大學籃球爭霸賽的比賽服裝有什麼規定？"
docs = db.similarity_search_with_score(query)
docs[0]

(Document(metadata={'author': 'User', 'creationDate': "D:20240312114435+08'00'", 'creator': 'Microsoft® Word 2016', 'file_path': '/content/basketball.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20240312114435+08'00'", 'page': 0, 'producer': 'Microsoft® Word 2016', 'source': '/content/basketball.pdf', 'subject': '', 'title': '', 'total_pages': 2, 'trapped': ''}, page_content='(9)比賽隊伍隊員比賽時應穿著運動鞋及運動服裝(如有背號之球衣為佳，如沒有體育室會提    \n          供號碼衣)出場比賽，嚴禁牛仔褲、襯衫及皮鞋，如服裝不符規定不得出賽。'),
 0.4050615429878235)

### 4.Retrival

##CPU

In [14]:
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


##CPU
#MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"
MODEL_NAME = "GeneZC/MiniChat-2-3B" # 較小的模型



# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True
        #device_map="auto"
        #device_map="cpu"
        ),
        #quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

'''
##GPU
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )
'''


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/749k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.04G [00:00<?, ?B/s]

'\n##GPU\nMODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"\n\n\n# 量化參數\nquantization_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_compute_dtype=torch.float16,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_use_double_quant=True)\n\n# llm 初始化\nllm = HuggingFacePipeline.from_model_id(\n    model_id=MODEL_NAME,\n    task="text-generation",\n    model_kwargs=dict(\n        torch_dtype=torch.float16,\n        trust_remote_code=True,\n        device_map="auto",\n        quantization_config=quantization_config),\n    pipeline_kwargs=dict(\n        max_new_tokens=1024,\n        temperature=0.0001,\n        top_p=0.95,\n        do_sample=True,\n        repetition_penalty=1.15) )\n'

In [15]:
# Prompt 模板
template = """
<s>
請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
{context}

{question}
[/INST]
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

result = qa_chain({"query": query})

# LLM 提供的回答
print("回覆：")
print(result["result"])

# 參考的相關資訊
print("參考資訊：")
print(result["source_documents"])

KeyboardInterrupt: 

## Advance
延伸 Basline 的內容，測試看看不同的 chunk 大小，對於生成結果的影響

In [ ]:
import numpy as np

In [ ]:
for chunk_size in np.append(50, np.arange(100, 1001, 100)):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=30)
  texts_chunks = text_splitter.split_documents(pages)

  # # 初始化 Embedding 模型
  # embedding_func = HuggingFaceEmbeddings(
  #   model_name="infgrad/stella-base-zh-v3-1792d",
  #   model_kwargs={"device": "cuda"},
  #   encode_kwargs={"normalize_embeddings": True})

  # load it into Chroma
  db = Chroma.from_documents(texts_chunks, embedding_func)

  # query it
  query = "請問臺北商業大學籃球爭霸賽的比賽服裝有什麼規定？"
  docs = db.similarity_search_with_score(query)

  # MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


  # # 量化參數
  # quantization_config = BitsAndBytesConfig(
  #     load_in_4bit=True,
  #     bnb_4bit_compute_dtype=torch.float16,
  #     bnb_4bit_quant_type="nf4",
  #     bnb_4bit_use_double_quant=True)

  # # llm 初始化
  # llm = HuggingFacePipeline.from_model_id(
  #     model_id=MODEL_NAME,
  #     task="text-generation",
  #     model_kwargs=dict(
  #         torch_dtype=torch.float16,
  #         trust_remote_code=True,
  #         device_map="auto",
  #         quantization_config=quantization_config),
  #     pipeline_kwargs=dict(
  #         max_new_tokens=1024,
  #         temperature=0.0001,
  #         top_p=0.95,
  #         do_sample=True,
  #         repetition_penalty=1.15) )

  # Prompt 模板
  template = """

  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  {context}

  {question}
  [/INST]
  """
  prompt = PromptTemplate(template=template, input_variables=["context", "question"])

  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

  result = qa_chain({"query": query})

  print(f'****************chunk_size: {chunk_size} ****************')
  # LLM 提供的回答
  print("回覆：")
  print(result["result"])

  # 參考的相關資訊
  print("參考資訊：")
  print(result["source_documents"])
  print('==='*80)